# DiffuseVAE in Google Colab

Google Colab에서 실행하는 예시

## Git Clone

Github에서 Personal Access Token 발급 후 Clone을 수행합니다. Token은 재사용이 가능하므로 안전한 곳에 저장해 두고 사용하면 됩니다.

### Personal Access Token 발급 방법

1. Github의 Settings - Developer settings (맨아래) 클릭
2. Personal access token 선택
3. Fine-grained tokens 또는 Tokens (classic) 선택
4. 이름, 만료일, 권한 등 설정 후 Generate token
    - Fine-grained tokens의 경우 최소 권한은 Contents만 Read and write로 선택
    - Classic의 경우 최소 권한은 repo만 선택
5. 키 복사 후 사용

In [ ]:
!git clone https://(발급 받은 Personal Access Token)@github.com/ysb06/dgm-2024-vae-diffusion.git

## 프로젝트 패키지 설치

In [ ]:
%pip install -e ..

## 예시: Baseline 학습 실행

### 하이퍼파라미터 로드

In [3]:
from hydra import initialize, compose
from omegaconf import DictConfig

def load_config() -> DictConfig:
    with initialize(config_path="../src/baseline/configs", version_base=None):
        config = compose(config_name="config")
    return config

config = load_config()
print(config)

{'dataset': {'ddpm': {'data': {'root': './datasets', 'name': 'cifar10', 'image_size': 32, 'hflip': True, 'n_channels': 3, 'norm': True}, 'model': {'dim': 128, 'attn_resolutions': '16,', 'n_residual': 2, 'dim_mults': '1,2,2,2', 'dropout': 0.3, 'n_heads': 8, 'beta1': 0.0001, 'beta2': 0.02, 'n_timesteps': 1000}, 'training': {'seed': 0, 'fp16': False, 'use_ema': True, 'z_cond': False, 'z_dim': 512, 'type': 'form1', 'ema_decay': 0.9999, 'batch_size': 32, 'epochs': 5000, 'log_step': 1, 'device': 'gpu:0', 'chkpt_interval': 1, 'optimizer': 'Adam', 'lr': 0.0002, 'restore_path': None, 'vae_chkpt_path': './outputs/vae.pt', 'results_dir': './outputs', 'workers': 2, 'grad_clip': 1.0, 'n_anneal_steps': 5000, 'loss': 'l2', 'chkpt_prefix': '', 'cfd_rate': 0.0}}, 'vae': {'data': {'root': './datasets', 'name': 'cifar10', 'image_size': 32, 'n_channels': 3, 'hflip': False}, 'model': {'enc_block_config': '32x7,32d2,32t16,16x4,16d2,16t8,8x4,8d2,8t4,4x3,4d4,4t1,1x3', 'enc_channel_config': '32:64,16:128,8:256

### Config 수정

In [8]:
# VSCode에서 실행하는 것과 다르게 Notebook파일이 있는 폴더가 현재 폴더 위치
config.dataset.ddpm.data.root = "../datasets"
config.dataset.ddpm.training.vae_chkpt_path = "../outputs/vae.pt"
config.dataset.ddpm.training.results_dir = "../outputs"
config.dataset.vae.data.root = "../datasets"
config.dataset.vae.training.results_dir = "../outputs"

config

{'dataset': {'ddpm': {'data': {'root': '../datasets', 'name': 'cifar10', 'image_size': 32, 'hflip': True, 'n_channels': 3, 'norm': True}, 'model': {'dim': 128, 'attn_resolutions': '16,', 'n_residual': 2, 'dim_mults': '1,2,2,2', 'dropout': 0.3, 'n_heads': 8, 'beta1': 0.0001, 'beta2': 0.02, 'n_timesteps': 1000}, 'training': {'seed': 0, 'fp16': False, 'use_ema': True, 'z_cond': False, 'z_dim': 512, 'type': 'form1', 'ema_decay': 0.9999, 'batch_size': 32, 'epochs': 5000, 'log_step': 1, 'device': 'gpu:0', 'chkpt_interval': 1, 'optimizer': 'Adam', 'lr': 0.0002, 'restore_path': None, 'vae_chkpt_path': '../outputs/vae.pt', 'results_dir': '../outputs', 'workers': 2, 'grad_clip': 1.0, 'n_anneal_steps': 5000, 'loss': 'l2', 'chkpt_prefix': '', 'cfd_rate': 0.0}}, 'vae': {'data': {'root': '../datasets', 'name': 'cifar10', 'image_size': 32, 'n_channels': 3, 'hflip': False}, 'model': {'enc_block_config': '32x7,32d2,32t16,16x4,16d2,16t8,8x4,8d2,8t4,4x3,4d4,4t1,1x3', 'enc_channel_config': '32:64,16:128,8

In [ ]:
import baseline.train_ae as vae_trainer
import baseline.train_ddpm as ddpm_trainer

vae_trainer.train(config)
ddpm_trainer.train(config)